# Setup


In [5]:
import pandas as pd
import numpy as np
import nltk
import spacy
import en_core_web_sm
from spacymoji import Emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pyvi import ViTokenizer, ViPosTagger
from underthesea import word_tokenize
from collections import Counter
import re


# Load data

In [33]:
data = pd.read_csv(r'/content/drive/MyDrive/speech/RAG/data_csv.csv')
data.head()


,file,content
0,Vịt Mốc Bình Định.txt,Vịt Mốc Bình Định\n\nVịt Mốc là giống vịt hướn...
1,Elliott Management Corporation.txt,Elliott Management Corporation\n\nElliott Mana...
2,Parma mccullochi.txt,Parma mccullochi\n\nParma mccullochi là một lo...
3,Dương Quang Thành.txt,Dương Quang Thành\n\nDương Quang Thành (sinh n...
4,Sergey Vadimovich Stepashin.txt,Sergey Vadimovich Stepashin\n\nSergei Vadimovi...


# Preprocessing

In [35]:
text = data['content'].to_list()

In [36]:
def remove_special_characters(data):
    # Sử dụng biểu thức chính quy để chỉ lấy các ký tự chữ cái và số
    clean_data = re.sub(r'[^\w\sÀ-ÿ]', ' ', data)
    return clean_data

In [37]:
remove_special_characters(text[0])

'Vịt Mốc Bình Định\n\nVịt Mốc là giống vịt hướng trứng  có màu lông nâu nhạt giống như mốc nên gợi là vịt Mốc  được hình thành do quá trình lai tạo tự nhiên và đã có từ lâu đời ở vùng Gò Bồi  huyện Tuy Phước  ở phía Đông của tỉnh Bình Định  Hiện nay  vịt có ở nhiều nơi của vùng duyên hải miền trung  \nĐặc điểm \nLúc mới nở  vịt con có hình dáng thon  dài  lông màu lang như bị mốc  mỏ và chân vàng nhạt  Khi trưởng thành  vịt có mình thon  ngực lép  đầu thon  cổ thon dài  hình dáng tạo với mặt đất 1 góc 45   lông màu nâu nhạt  vàng có đốm nâu  vàng hoe và vàng  ở con trống có 1   2 lông móc  mỏ và chân có màu xám đen  chì   đỏ hoặc vàng  khối lượng 1 4   1 5\xa0kg con  Khả năng sản xuất  Vịt đẻ bói lúc 140 ngày tuổi  năng suất trứng đạt 90 quả 25 tuần  khối lượng trứng đạt 55 gram quả \n\n'

In [38]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


# tạo ra một từ điển (dic) để ánh xạ các ký tự tiếng Việt từ bảng mã Latin-1 (1252) sang UTF-8.
def loaddicchar():
  dic = {}
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
  charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
  for i in range(len(char1252)):
      dic[char1252[i]] = charutf8[i]
  return dic

dicchar = loaddicchar()

# Đưa dữ liệu qua hàm này để chuẩn hóa lại
def convert_unicode(data):
    return re.sub(r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], data)

In [39]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def vn_word_to_telex_type(word):
    dau_cau = 0
    new_word = ''
    for char in word:
        # Nếu char có trong từ điển nguyen_am_to_ids, thì biến x sẽ nhận giá trị được ánh xạ từ char và biến y sẽ nhận giá trị thứ hai trong cặp giá trị tương ứng với char trong từ điển.
        # Nếu char không tồn tại trong từ điển, thì get() sẽ trả về giá trị mặc định là (-1, -1).
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            new_word += char
            continue
        if y != 0:
            dau_cau = y
        new_word += bang_nguyen_am[x][-1]
    new_word += bang_ky_tu_dau[dau_cau]
    return new_word

def vn_sentence_to_telex_type(sentence):
    """
    Chuyển câu tiếng việt có dấu về kiểu gõ telex.
    :param sentence:
    :return:
    """
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = vn_word_to_telex_type(word)
    return ' '.join(words)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)

    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

# kiểm tra xem một từ tiếng Việt có tuân theo quy tắc đặt cách nhau của các nguyên âm không
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = chuan_hoa_dau_tu_tieng_viet(word)
    return ' '.join(words)


In [40]:
def lower_case(data):
  return data.lower()

def remove_dub(sentence):
    return re.sub(r'([A-Z])\1+', lambda m: m.group(1), sentence, flags=re.IGNORECASE)

In [41]:
nlp = en_core_web_sm.load()
emoji = Emoji(nlp)
nlp.add_pipe('emoji', first=True)

def remove_emoji(text):
  doc = nlp(text)
  text_with_spaces = ''.join(token.text_with_ws if not token._.is_emoji else ' ' for token in doc)
  return text_with_spaces

In [42]:
teencode_data = pd.read_csv('/content/drive/MyDrive/BA documents/DBA/Data/teencode.txt',names=['teencode','map'],sep='\t',)
teencode_list = teencode_data['teencode'].to_list()
map_list = teencode_data['map'].to_list()

In [43]:
def searchTeencode(word):
  try:
    global teencode_count
    index = teencode_list.index(word)
    map_word = map_list[index]
    teencode_count += 1
    return map_word
  except:
    pass

stopword_count = 0
teencode_count =0
def Teencode(sentence):
    # Xứ lý các từ ok
    sentence = re.sub(r'\bok\w*\b', 'ok', sentence)
    # Tokenize
    List_tokens = word_tokenize(sentence,format='text')
    List_tokens = word_tokenize(sentence)

    # Teencode
    for tokens_idx, text_tokens in enumerate(List_tokens):
        deteencoded = searchTeencode(text_tokens)
        if deteencoded is not None:
            List_tokens[tokens_idx] = deteencoded

    deteencode_sentence = " ".join(List_tokens)
    return deteencode_sentence


In [44]:

def text_preprocess(document):
    # xóa ký tự đặc biệt
    document = remove_special_characters(document)
    # Xóa emoji
    document = remove_emoji(document)
    # đưa về lower
    document = document.lower()
    # Xóa ký tự cố ý viết dài
    document = remove_dub(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # chuẩn hóa teencode
    document = Teencode(document)
    # tách từ
    document = ViTokenizer.tokenize(document)
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [45]:
document = 'máy đẹp xuất sắc, mua onl mỗi tội tiki hôk bán kèm cường lực và ốp hjhj =))))) máy đẹp lứm nên mua nka 🥺😻😽'
document = text_preprocess(document)
print(document)

máy đẹp xuất_sắc mua online mỗi tội tiki không bán kèm cường_lực và ốp hi_hi máy đẹp lứm nên mua nha


In [46]:
data['processed'] = data['content'].apply(text_preprocess)

In [47]:
stopwords = ['có', 'thì', 'nên', 'và', 'là', 'này', 'bị', 'cũng', 'cho', 'đã', 'của', 'mà', 'lên', 'như', 'với', 'sẽ', 'lại', 'còn', 'luôn', 'rồi', 'nha', 'nhé', 'ạ', 'à', 'ơi', 'mình', 'vẫn', 'về', 'vì', '*', 'kiểu', 'thấy', 'á', 'lắm']

In [48]:
def word_segmentation(x):
    word=word_tokenize(x)
    return word

def stopword_find(x):
      return Counter(x)

def remove_stopwords(line):
  s2=[]
  for word in line:
      if word not in stopwords:
          s2.append(word)
  return s2
def detokenize(tokens):
    # Join tokens to form a complete sentence
    return ' '.join(tokens)

In [49]:
data["tokenized_text"]=data["processed"].apply(word_segmentation)
data["tokenized_text"]=data["tokenized_text"].apply(remove_stopwords)
data['processed'] = data['tokenized_text'].apply(detokenize)

In [50]:
data['processed']

0        vịt mốc bình định vịt mốc giống vịt hướng trứn...
1        eliot management corporation eliot management ...
2        parma mculochi parma mculochi một loài cá biển...
3        dương quang thành dương quang thành sinh ngày ...
4        sergey vadimovich stepashin sergei vadimovich ...
                               ...                        
10001    hồ thị thanh hồng hồ thị thanh hồng sinh 25 th...
10002    vòng chung_kết giải bóng_đá trong nhà vô_địch ...
10003    sarmizegetusa regia sarmizegetusa regia làsarm...
10004    chiên ngập dầu chiên ngập dầu một phương_pháp ...
10005    takabuti takabuti một phụ_nữ kết_hôn đạt đến đ...
Name: processed, Length: 10006, dtype: object

In [ ]:
data = data[data['transcript_processed'].notnull()]

In [51]:
data.head()

,file,content,processed,tokenized_text
0,Vịt Mốc Bình Định.txt,Vịt Mốc Bình Định\n\nVịt Mốc là giống vịt hướn...,vịt mốc bình định vịt mốc giống vịt hướng trứn...,"[vịt, mốc, bình định, vịt, mốc, giống, vịt, hư..."
1,Elliott Management Corporation.txt,Elliott Management Corporation\n\nElliott Mana...,eliot management corporation eliot management ...,"[eliot, management, corporation, eliot, manage..."
2,Parma mccullochi.txt,Parma mccullochi\n\nParma mccullochi là một lo...,parma mculochi parma mculochi một loài cá biển...,"[parma, mculochi, parma, mculochi, một, loài, ..."
3,Dương Quang Thành.txt,Dương Quang Thành\n\nDương Quang Thành (sinh n...,dương quang thành dương quang thành sinh ngày ...,"[dương quang, thành, dương quang, thành, sinh,..."
4,Sergey Vadimovich Stepashin.txt,Sergey Vadimovich Stepashin\n\nSergei Vadimovi...,sergey vadimovich stepashin sergei vadimovich ...,"[sergey, vadimovich, stepashin, sergei, vadimo..."


In [52]:
data.to_csv('/content/drive/MyDrive/speech/RAG/clean_csv.csv', index=False, encoding = 'utf-8-sig')